<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/Bot_Funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install ta
!pip install requests

In [37]:
import yfinance as yf
import numpy as np
import pandas as pd
import ta
import requests
from sklearn.ensemble import RandomForestClassifier

# 1️⃣ Configurações
ASSETS = [
    "BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "XRP-USD",
    "ADA-USD", "AVAX-USD", "DOGE-USD", "DOT-USD", "SHIB-USD",
    "MATIC-USD", "TRX-USD", "LINK-USD", "LTC-USD", "ATOM-USD"
]
TELEGRAM_TOKEN = "8044593190:AAFtUWYHd3uqd-AtQi3uqg42F9G6uV95v8k"
TELEGRAM_CHAT_ID = "-4744645054"

# 2️⃣ Função para obter dados (corrigida)
def get_stock_data(asset, interval="15m", period="14d"):
    data = yf.download(asset, period=period, interval=interval, progress=False)

    # 🔧 Corrige colunas se vierem com MultiIndex
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # 🔧 Remove nome do ativo das colunas (ex: "NVDA Close" → "Close")
    data.columns = [col.split()[-1] if " " in col else col for col in data.columns]

    # 🔧 Renomeia colunas padrão, se necessário
    col_map = {}
    for col in data.columns:
        for std_col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
            if std_col.lower() in col.lower():
                col_map[col] = std_col
    data = data.rename(columns=col_map)

    # Verifica colunas essenciais
    required = ["Open", "High", "Low", "Close", "Volume"]
    if not all(col in data.columns for col in required):
        raise ValueError(f"⚠️ Dados de {asset} não possuem todas as colunas necessárias.")

    return data

def calculate_indicators(data):
    data = data.copy()
    data.reset_index(drop=True, inplace=True)

    # 🧹 Garante que são Series 1D com float
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        # Garante 1D com dtype correto
        values = np.ravel(data[col].to_numpy()).astype("float64")
        data[col] = pd.Series(values, index=data.index, name=col)

        # Debug seguro
        print(f"✅ DEBUG: {col} | type = {type(data[col])}, shape = {data[col].shape}, dtype = {data[col].dtypes if hasattr(data[col], 'dtypes') else data[col].dtype}")

    # Indicadores técnicos
    data["RSI"] = ta.momentum.RSIIndicator(close=data["Close"], window=14).rsi()
    sma_50 = ta.trend.SMAIndicator(close=data["Close"], window=50)
    data["SMA_50"] = sma_50.sma_indicator()
    sma_200 = ta.trend.SMAIndicator(close=data["Close"], window=200)
    data["SMA_200"] = sma_200.sma_indicator()
    macd = ta.trend.MACD(close=data["Close"])
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()
    bb = ta.volatility.BollingerBands(close=data["Close"], window=20)
    data["Bollinger_Upper"] = bb.bollinger_hband()
    data["Bollinger_Lower"] = bb.bollinger_lband()
    adx = ta.trend.ADXIndicator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["ADX"] = adx.adx()
    stoch = ta.momentum.StochasticOscillator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["Stoch_K"] = stoch.stoch()
    data["Stoch_D"] = stoch.stoch_signal()

    # VWAP manual
    data["TP"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["VWAP"] = (data["TP"] * data["Volume"]).cumsum() / (data["Volume"].replace(0, np.nan).cumsum())
    data.drop("TP", axis=1, inplace=True)

    # Padrões de candle
    data["Doji"] = ((abs(data["Close"] - data["Open"]) / (data["High"] - data["Low"] + 1e-9)) < 0.1).astype(int)
    data["Engulfing"] = (
        (data["Open"].shift(1) > data["Close"].shift(1)) &
        (data["Open"] < data["Close"]) &
        (data["Close"] > data["Open"].shift(1)) &
        (data["Open"] < data["Close"].shift(1))
    ).astype(int)
    data["Hammer"] = (
        ((data["High"] - data["Low"]) > 3 * abs(data["Open"] - data["Close"])) &
        ((data["Close"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6) &
        ((data["Open"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6)
    ).astype(int)

    data.dropna(inplace=True)
    return data



# 4️⃣ Treinar modelo
def train_ml_model(data):
    if len(data) < 50:
        return None

    df = data.copy()
    df["Signal"] = np.where((df["SMA_50"] > df["SMA_200"]) & (df["RSI"] < 30), 1, 0)

    features = get_feature_columns()
    X = df[features]
    y = df["Signal"]

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

# 🔎 Colunas de features (evita repetição)
def get_feature_columns():
    return [
        "RSI", "MACD", "MACD_Signal", "SMA_50", "SMA_200", "Bollinger_Upper",
        "Bollinger_Lower", "ADX", "Stoch_K", "Stoch_D", "VWAP",
        "Doji", "Engulfing", "Hammer"
    ]
def generate_explanation(row, prediction):
    explanation = []

    if prediction == 1:
        explanation.append("🟢 O modelo prevê uma tendência de ALTA.")
    else:
        explanation.append("🔴 O modelo prevê uma tendência de BAIXA.")

    if row["RSI"] < 30:
        explanation.append("🔽 RSI abaixo de 30 indica sobrevenda.")
    elif row["RSI"] > 70:
        explanation.append("🔼 RSI acima de 70 indica sobrecompra.")

    if row["SMA_50"] > row["SMA_200"]:
        explanation.append("📈 SMA 50 acima da 200, tendência de alta.")
    else:
        explanation.append("📉 SMA 50 abaixo da 200, tendência de baixa.")

    if row["MACD"] > row["MACD_Signal"]:
        explanation.append("💹 MACD cruzando para cima, possível reversão positiva.")
    else:
        explanation.append("🔻 MACD abaixo da linha de sinal.")

    if row["Doji"] == 1:
        explanation.append("⚠️ Padrão de candle Doji detectado (possível reversão).")

    if row["Engulfing"] == 1:
        explanation.append("📊 Padrão de engolfo detectado (sinal forte de reversão).")

    return "\n".join(explanation)

def calculate_targets(current_price, direction, atr=0.02):
    if direction == 1:
        return {
            "TP1": round(current_price * (1 + atr), 2),
            "TP2": round(current_price * (1 + atr * 2), 2),
            "SL": round(current_price * (1 - atr), 2)
        }
    else:
        return {
            "TP1": round(current_price * (1 - atr), 2),
            "TP2": round(current_price * (1 - atr * 2), 2),
            "SL": round(current_price * (1 + atr), 2)
        }

# 5️⃣ Rodar análise
def run_analysis():
    results = {}

    for asset in ASSETS:
        print(f"📊 Analisando {asset}...")

        try:
            data_15m = calculate_indicators(get_stock_data(asset, "15m", "14d"))
            data_1h = calculate_indicators(get_stock_data(asset, "1h", "30d"))
        except Exception as e:
            print(f"❌ Erro ao processar {asset}: {e}")
            continue

        model_15m = train_ml_model(data_15m)
        model_1h = train_ml_model(data_1h)

        if model_15m is None or model_1h is None:
            print(f"⚠️ Dados insuficientes para {asset}.")
            continue

        features = get_feature_columns()
        latest_15m_input = pd.DataFrame([data_15m[features].iloc[-1]])
        latest_1h_input = pd.DataFrame([data_1h[features].iloc[-1]])

        pred_15m = model_15m.predict(latest_15m_input)[0]
        pred_1h = model_1h.predict(latest_1h_input)[0]

        prob_15m = model_15m.predict_proba(latest_15m_input)[0][pred_15m]
        prob_1h = model_1h.predict_proba(latest_1h_input)[0][pred_1h]

        results[asset] = {"15m": pred_15m, "1h": pred_1h}

        # Geração de explicação detalhada
        latest_15m = data_15m.iloc[-1]
        latest_1h = data_1h.iloc[-1]
        current_price = latest_15m["Close"]

        targets_15m = calculate_targets(current_price, pred_15m)

        explanation_15m = generate_explanation(latest_15m, pred_15m)
        explanation_1h = generate_explanation(latest_1h, pred_1h)

        message = f"""🔍 {asset}

💰 Preço atual: ${current_price:.2f}

⏱️ 15 minutos:
{'🟢 COMPRA' if pred_15m == 1 else '🔴 VENDA'} com {prob_15m*100:.1f}% de confiança
🎯 TP1: ${targets_15m['TP1']} | TP2: ${targets_15m['TP2']} | SL: ${targets_15m['SL']}
{explanation_15m}

⏱️ 1 hora:
{'🟢 COMPRA' if pred_1h == 1 else '🔴 VENDA'} com {prob_1h*100:.1f}% de confiança
{explanation_1h}
"""
        send_telegram_message(message)
        print(message)

# 6️⃣ Executar
if __name__ == "__main__":
    input("🚀 Pressione Enter para iniciar a análise...\n")
    run_analysis()


🚀 Pressione Enter para iniciar a análise...

📊 Analisando BTC-USD...
✅ DEBUG: Open | type = <class 'pandas.core.series.Series'>, shape = (1326,), dtype = float64
✅ DEBUG: High | type = <class 'pandas.core.series.Series'>, shape = (1326,), dtype = float64
✅ DEBUG: Low | type = <class 'pandas.core.series.Series'>, shape = (1326,), dtype = float64
✅ DEBUG: Close | type = <class 'pandas.core.series.Series'>, shape = (1326,), dtype = float64
✅ DEBUG: Volume | type = <class 'pandas.core.series.Series'>, shape = (1326,), dtype = float64
✅ DEBUG: Open | type = <class 'pandas.core.series.Series'>, shape = (716,), dtype = float64
✅ DEBUG: High | type = <class 'pandas.core.series.Series'>, shape = (716,), dtype = float64
✅ DEBUG: Low | type = <class 'pandas.core.series.Series'>, shape = (716,), dtype = float64
✅ DEBUG: Close | type = <class 'pandas.core.series.Series'>, shape = (716,), dtype = float64
✅ DEBUG: Volume | type = <class 'pandas.core.series.Series'>, shape = (716,), dtype = float64
🔍 